In [1]:
!pip install torch torchdata
!pip install transformers datasets
!pip install evaluate rouge_score loralib peft
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=414b9c7a7740ddd6d7ec163b5d65d8f116bf4bebbd22a697daf57a67d6a9676f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from datasets import load_dataset
import transformers
import accelerate
import torch


print(transformers.__version__)
print(accelerate.__version__)

4.35.2
0.24.1


In [3]:
val_data=load_dataset("scientific_papers",'arxiv',split='validation[:1%]')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [4]:
model_name='facebook/bart-large-cnn'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
val_data['article'][0]

"the interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices .\nthe twisted nematic liquid crystal cell serves as an example .\nit consists of a nematic liquid crystal confined between two parallel walls , both providing homogeneous planar anchoring but with mutually perpendicular easy directions . in this case\nthe orientation of the nematic director is tuned by the application of an external electric or magnetic field .\na precise control of the surface alignment extending over large areas is decisive for the functioning of such devices .\nmost studies have focused on nematic liquid crystals in contact with laterally uniform substrates . on the other hand substrate inhomogeneities\narise rather naturally as a result of surface treatments such as rubbing .\nthus the nematic texture near the surface is in fact non - uniform .\nthis non - uniformity , however , is smeared out beyon

In [6]:
val_new=val_data.filter(lambda example, index: index % 10 == 0, with_indices=True)

Filter:   0%|          | 0/64 [00:00<?, ? examples/s]

In [7]:
torch.device("cuda")

device(type='cuda')

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline

config = PeftConfig.from_pretrained("cherryberry01/Re_Sum")
peft_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
peft_model = PeftModel.from_pretrained(peft_model, "cherryberry01/Re_Sum")
tokenizer=AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

original_model_summaries = []
manual_model_summaries = []
peft_model_summaries = []

for idx, text in enumerate(val_data['article']):
  human_baseline_text_output = val_data['abstract'][idx]

  original_sum=pipeline('summarization',model=original_model,tokenizer=tokenizer,truncation=True)
  original_summary=original_sum(text,min_length=10,max_length=500)
  peft_sum=pipeline('summarization',model=peft_model,tokenizer=tokenizer,truncation=True)
  peft_summary=peft_sum(text,min_length=10,max_length=500)

  original_model_summaries.append(original_summary[0]['summary_text'])
  manual_model_summaries.append(human_baseline_text_output)
  peft_model_summaries.append(peft_summary[0]['summary_text'])

zipped_summaries = list(zip(manual_model_summaries, original_model_summaries, peft_model_summaries))


In [10]:
zipped_summaries = list(zip(manual_model_summaries, original_model_summaries, peft_model_summaries))
zipped_summaries

[(' we study the phase behavior of a nematic liquid crystal confined between a flat substrate with strong anchoring and a patterned substrate whose structure and local anchoring strength we vary . by first evaluating an effective surface free energy function characterizing the patterned substrate we derive an expression for the effective free energy of the confined nematic liquid crystal . \n then we determine phase diagrams involving a homogeneous state in which the nematic director is almost uniform and a hybrid aligned nematic state in which the orientation of the director varies through the cell . \n direct minimization of the free energy functional were performed in order to test the predictions of the effective free energy method . \n we find remarkably good agreement between the phase boundaries calculated from the two approaches . \n in addition the effective energy method allows one to determine the energy barriers between two states in a bistable nematic device . ',
  'Nemati

In [12]:
import pandas as pd
df = pd.DataFrame(zipped_summaries, columns = ['manual_model_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

,manual_model_summaries,original_model_summaries,peft_model_summaries
0,we study the phase behavior of a nematic liqu...,Nematic liquid crystals in contact with non - ...,Nematic liquid crystals in contact with non - ...
1,determining the scaling relations between gal...,Clusters were first detected as early as the 1...,galaxy clusters are the largest peaks in the ...
2,we show how to control spatial quantum correl...,quantum correlations between components of a ...,quantum correlations between components of a ...
3,class i methanol masers are believed to be p...,Methanol masers are often found in star - form...,Methanol masers are often found in star - form...
4,"nowadays , scientific challenges usually requ...",Interdisciplinary research has recently gained...,Interdisciplinary research has recently gained...
5,we present a theoretical study of the infrare...,In this paper we discuss corresponding theoret...,This paper presents theoretical predictions fo...
6,two questions that naturally arise in @xmath0...,In simulations of stellar systems the gravitat...,In stellar systems the gravitational interacti...
7,we study a holomorphic representation for spi...,spinfoams @xcite provide a covariant formulati...,spinfoams @xcite provide a covariant formulat...


In [14]:
import evaluate
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=manual_model_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=manual_model_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
print('Original Model Rogue Score:')
print(original_model_results)
print('PEFT Model Rogue Score:')
print(peft_model_results)

Original Model Rogue Score:
{'rouge1': 0.2863152254443702, 'rouge2': 0.08580960861379738, 'rougeL': 0.17532470918501186, 'rougeLsum': 0.23256669647122025}
PEFT Model Rogue Score:
{'rouge1': 0.31574330832996994, 'rouge2': 0.09234086493786058, 'rougeL': 0.18540422674904494, 'rougeLsum': 0.24658712676340572}
